In [1]:
import os

In [2]:
pwd

'd:\\PW_DS\\NLP\\End-To-End-Text-Summarizer-Project-using-Transformer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\PW_DS\\NLP\\End-To-End-Text-Summarizer-Project-using-Transformer'

## Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


## Config Manager

In [7]:
from Text_Summarizer.constants import  *
from Text_Summarizer.utils.common import read_yaml, create_directories

In [8]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [9]:
PARAMS_FILE_PATH

WindowsPath('params.yaml')

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation

        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config
        

## Components

In [21]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install --upgrade tokenizers


Note: you may need to restart the kernel to use updated packages.


In [24]:
# pip install -r requirements.txt --force-reinstall
!pip install protobuf

  Using cached protobuf-4.25.2-cp38-cp38-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.2-cp38-cp38-win_amd64.whl (413 kB)


In [25]:
import os
from Text_Summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


In [30]:
class DataTransformation:
    def __init__(
        self,
        config: DataTransformationConfig):

        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name) 



    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=130, truncation=True)

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask' : input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }



    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum-dataset'))


In [31]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e


[2024-01-11 17:46:15,562: INFO: common: yaml file: config\config.yaml loaed successfully.]
[2024-01-11 17:46:15,566: INFO: common: yaml file: params.yaml loaed successfully.]
[2024-01-11 17:46:15,567: INFO: common: directory: artifacts created successfully.]
[2024-01-11 17:46:15,569: INFO: common: directory: artifacts/data_transformation created successfully.]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Maaz\anaconda3\envs\venv\lib\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 28702.63 examples/s]
